In [1]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

    
val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


[label: int, tweet: string]

In [3]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.feature.{HashingTF, Tokenizer, StringIndexer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val rf = new RandomForestClassifier()
    .setLabelCol("label")
    .setFeaturesCol(hashingTF.getOutputCol)
    .setNumTrees(10)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, rf))


tokenizer = tok_ca909d060370
hashingTF = hashingTF_bb790763e44b
rf = rfc_9d7cc9266cf0
pipeline = pipeline_f1fbeb79703f


pipeline_f1fbeb79703f

In [4]:
val model = pipeline.fit(raw_sentiment)

model = pipeline_f1fbeb79703f


pipeline_f1fbeb79703f

In [5]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [6]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel = pipeline_f1fbeb79703f


pipeline_f1fbeb79703f

In [7]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[7,14,21,54...|[4.40571171280962...|[0.44057117128096...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[170,193,22...|[5.03207367084828...|[0.50320736708482...|       0.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[10,36,77,1...|[5.64508367805877...|[0.56450836780587...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[82,191,296...|[5.62272968131892...|[0.56227296813189...|       0.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[18,96,130,...|[5.03034849862725...|[0.50303484986272...|       0.0|
|    0|@

predictionsDF = [label: int, tweet: string ... 5 more fields]


[label: int, tweet: string ... 5 more fields]

In [8]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [9]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show

+-------------------+
|  clean_probability|
+-------------------+
| 0.5594288287190375|
| 0.4967926329151717|
|0.43549163219412207|
| 0.4377270318681079|
| 0.4969651501372742|
| 0.5144744106785156|
| 0.5144744106785156|
|  0.549145505493379|
| 0.5079832589933295|
| 0.5082814404653254|
| 0.5144744106785156|
| 0.4377270318681079|
|0.46344600013957865|
| 0.5786196440412914|
|0.48792752220848357|
| 0.5227077113131553|
| 0.5032428419783137|
| 0.5144744106785156|
|  0.536445520749846|
| 0.5471666853609675|
+-------------------+
only showing top 20 rows

